In [1]:
import subprocess

#Params
datasetPath = "../datasets/"
dataPath = "./data/"
#resultPath = "./results/heatmaps/Apache/"
filenames = ["Apache"]
perf="perf"


#Params for sensistivity
NBINS = 40 # Number of vertical bins for threshold
NSUBS = 10 # Number of training sets to average on
srm = 1 # Minimum sampling size
srM=[]
srs = []
for k,filename in enumerate(filenames):
    srM.append(int(subprocess.check_output("echo $(wc -l < "+datasetPath+filename+".csv)", shell=True))) # Maximum sampling size
    srs.append(srM[k]//100) # Sampling step between two iterations
    
    
#Params for Decision Tree
#Default params
treeParamsDefault = {
    "criterion":"gini",
    "splitter":"best",
    "max_features":None,
    "max_depth":None,
    "min_samples_split":2,
    "min_samples_leaf":1,
    "min_weight_fraction_leaf":0.,
    "max_leaf_nodes":None,
    "class_weight":None,
    "random_state":None,
    "min_impurity_decrease":1e-7,
    "presort":False
}
treeParams = treeParamsDefault

In [43]:
import sys
import pandas as pd
from sklearn import tree
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error as mse

In [27]:
data = pd.read_csv("../datasets/Apache.csv")

In [28]:
shuffle_split = ShuffleSplit(train_size=20, n_splits=10)

/usr/lib64/python3.6/site-packages/sklearn/model_selection/_split.py:1639: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [31]:
c = tree.DecisionTreeRegressor()

In [32]:
for train_index, test_index in shuffle_split.split(data,data.perf):
    c.fit(data.drop(["perf"],axis=1).iloc[train_index], data.perf.iloc[train_index])
    pred = c.predict(data.drop(["perf"],axis=1).iloc[test_index])

In [33]:
pred

array([2460., 1170.,  870., 2190., 1200., 1470., 1470., 2640., 1200.,
       1110., 1350., 2640.,  870., 1800., 1350., 1170.,  870., 1200.,
       1950., 1350.])

In [35]:
data['perf'].iloc[test_index]

89     2220
112    1200
95     1050
164    2400
34     1140
167    1380
158    1260
113    2040
46     1230
39     1200
84     1140
149    1920
87      960
118    1890
81     1170
97     1050
76      900
100    1230
178    2280
53     1230
Name: perf, dtype: int64

In [36]:
ndf = pd.DataFrame()

In [39]:
ndf['perf'] = data['perf'].iloc[test_index]
ndf['pred'] = pred
ndf['error'] = ndf['perf'] - ndf['pred']
ndf['square_error'] = ndf['error'] * ndf['error']

In [41]:
ndf['square_error'].mean()

65520.0

In [42]:
ndf

,perf,pred,error,square_error
89,2220,2460.0,-240.0,57600.0
112,1200,1170.0,30.0,900.0
95,1050,870.0,180.0,32400.0
164,2400,2190.0,210.0,44100.0
34,1140,1200.0,-60.0,3600.0
167,1380,1470.0,-90.0,8100.0
158,1260,1470.0,-210.0,44100.0
113,2040,2640.0,-600.0,360000.0
46,1230,1200.0,30.0,900.0
39,1200,1110.0,90.0,8100.0


In [44]:
mse(ndf['perf'],ndf['pred'])

65520.0